In [8]:
import numpy as np
import random
from datetime import datetime

# Dicionários de apoio
dicio = {
    '1,1': 0, '1,2': 1, '1,3': 2, '1,4': 3, '1,5': 4,
    '2,1': 5, '2,2': 6, '2,3': 7, '2,4': 8, '2,5': 9,
    '3,1': 10, '3,2': 11, '3,3': 12, '3,4': 13, '3,5': 14,
    '4,1': 15, '4,2': 16, '4,3': 17, '4,4': 18, '4,5': 19,
    '5,1': 20, '5,2': 21, '5,3': 22, '5,4': 23, '5,5': 24
}

# Posições inversas para traduzir índices para coordenadas
dicio_inverso = {v: k for k, v in dicio.items()}

# Posições na borda (as únicas que podem ser movidas de acordo com as regras do Quixo)
posicoes_borda = np.array([
    '1,1', '1,2', '1,3', '1,4', '1,5',
    '2,1', '2,5', '3,1', '3,5',
    '4,1', '4,5', '5,1', '5,2', '5,3', '5,4', '5,5'
])

dicioLinhas = {
    'Linha 1': [0, 1, 2, 3, 4], 'Linha 2': [5, 6, 7, 8, 9], 'Linha 3': [10, 11, 12, 13, 14],
    'Linha 4': [15, 16, 17, 18, 19], 'Linha 5': [20, 21, 22, 23, 24],
    'Coluna 1': [0, 5, 10, 15, 20], 'Coluna 2': [1, 6, 11, 16, 21],
    'Coluna 3': [2, 7, 12, 17, 22], 'Coluna 4': [3, 8, 13, 18, 23],
    'Coluna 5': [4, 9, 14, 19, 24], 'Diagonal 1': [0, 6, 12, 18, 24],
    'Diagonal 2': [4, 8, 12, 16, 20]
}

# Classe para armazenar o histórico de jogadas
class HistoricoJogadas:
    def __init__(self, max_jogadas=10):
        self.jogadas = []
        self.max_jogadas = max_jogadas

    def adicionar_jogada(self, jogador, origem, destino):
        timestamp = datetime.now().strftime("%H:%M:%S")
        jogada = {
            "jogador": jogador,
            "origem": origem,
            "destino": destino,
            "timestamp": timestamp
        }
        self.jogadas.append(jogada)
        if len(self.jogadas) > self.max_jogadas:
            self.jogadas.pop(0)

    def obter_ultimas_jogadas(self, quantidade=None):
        if quantidade is None or quantidade > len(self.jogadas):
            return self.jogadas
        return self.jogadas[-quantidade:]

    def imprimir_historico(self):
        print("\n=== HISTÓRICO DE JOGADAS ===")
        for i, jogada in enumerate(self.jogadas, 1):
            print(f"{i}. {jogada['jogador']} moveu de {jogada['origem']} para {jogada['destino']} ({jogada['timestamp']})")
        print("===========================\n")

# Classe do jogo
class Jogo:
    def __init__(self, tabuleiro=None, jogador='🔴'):  # Usuário (🔴) agora inicia
        self.tabuleiro = np.full(25, "⬜", dtype=str) if tabuleiro is None else np.array(tabuleiro, dtype=str)
        self.jogador_atual = jogador
        self.historico = HistoricoJogadas()
        self.ultima_jogada_agente = None

    def turno(self):
        return self.jogador_atual

    # Método que verifica se há uma linha, coluna ou diagonal vencedora
    def venceu(self):
        for nome, indices in dicioLinhas.items():
            valores = [self.tabuleiro[i] for i in indices]
            if all(v == "❌" for v in valores) or all(v == "🔴" for v in valores):
                return True
        return False

    # Método que verifica se o jogo deu empate (nenhum vencedor e o tabuleiro está cheio)
    def empate(self):
        # Verifica se não há vitória e se não existem mais espaços vazios no tabuleiro
        return not self.venceu() and "⬜" not in self.tabuleiro

    # Método que calcula a utilidade do estado do jogo para um jogador
    def calcular_utilidade(self, jogador):
        if self.venceu():
            # Jogador atual é quem acabou de mover, então se venceu, a utilidade é 1 para ele
            return 1 if self.jogador_atual == jogador else -1

        # Se não venceu, verificamos vantagens parciais
        pontos = 0

        # Conta quantas peças de cada tipo estão presentes em cada linha/coluna/diagonal
        for nome, indices in dicioLinhas.items():
            valores = [self.tabuleiro[i] for i in indices]
            if jogador == "❌":
                pontos += valores.count("❌") * 0.1
                pontos -= valores.count("🔴") * 0.1
            else:
                pontos += valores.count("🔴") * 0.1
                pontos -= valores.count("❌") * 0.1

        return pontos

    def jogos_validos(self):
        filhos = []
        for pos in posicoes_borda:
            i = dicio[pos]
            # Seguindo as regras do Quixo: só pode mover peças próprias ou vazias
            if self.tabuleiro[i] == "⬜" or self.tabuleiro[i] == self.jogador_atual:
                for destino in self.movimentos_validos(pos):
                    novo = self.jogar((pos, destino))
                    if novo:
                        filhos.append((pos, destino))
        return filhos

    def jogar(self, movimento):
        origem_str, destino_str = movimento
        origem_i, origem_j = map(int, origem_str.split(','))
        destino_i, destino_j = map(int, destino_str.split(','))

        if origem_str not in dicio or destino_str not in dicio:
            return None

        origem = dicio[origem_str]
        destino = dicio[destino_str]

        # Verifica se a posição está na borda (regra do Quixo)
        if origem_str not in posicoes_borda:
            return None

        # Verifica se a origem tem uma peça do jogador atual ou está vazia
        if self.tabuleiro[origem] != "⬜" and self.tabuleiro[origem] != self.jogador_atual:
            return None

        novo_tabuleiro = np.array(self.tabuleiro)
        if origem_i == destino_i:  # linha
            linha = origem_i
            indices = [dicio[f"{linha},{j}"] for j in range(1, 6)]
            if destino_j > origem_j:
                for k in range(0, 4):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k + 1]]
                novo_tabuleiro[indices[4]] = self.jogador_atual
            else:
                for k in range(4, 0, -1):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k - 1]]
                novo_tabuleiro[indices[0]] = self.jogador_atual
        elif origem_j == destino_j:  # coluna
            coluna = origem_j
            indices = [dicio[f"{i},{coluna}"] for i in range(1, 6)]
            if destino_i > origem_i:
                for k in range(0, 4):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k + 1]]
                novo_tabuleiro[indices[4]] = self.jogador_atual
            else:
                for k in range(4, 0, -1):
                    novo_tabuleiro[indices[k]] = novo_tabuleiro[indices[k - 1]]
                novo_tabuleiro[indices[0]] = self.jogador_atual
        else:
            return None

        proximo_jogador = "🔴" if self.jogador_atual == "❌" else "❌"
        novo_jogo = Jogo(novo_tabuleiro, proximo_jogador)
        novo_jogo.historico = self.historico  # Compartilha o histórico
        return novo_jogo

    def movimentos_validos(self, origem_str):
        origem_i, origem_j = map(int, origem_str.split(','))
        destinos = []

        # No Quixo, as peças só podem ser empurradas da borda para o lado oposto
        if origem_i == 1:  # Primeira linha
            destinos.append(f"5,{origem_j}")
        elif origem_i == 5:  # Última linha
            destinos.append(f"1,{origem_j}")

        if origem_j == 1:  # Primeira coluna
            destinos.append(f"{origem_i},5")
        elif origem_j == 5:  # Última coluna
            destinos.append(f"{origem_i},1")

        return destinos

    def imprimir(self):
      return print("| " + self.tabuleiro[0] + " | " + self.tabuleiro[1] + " | " + self.tabuleiro[2]+ " | " + self.tabuleiro[3] + " | " + self.tabuleiro[4] + " | " + "\n" +
      "| " + self.tabuleiro[5] + " | " + self.tabuleiro[6] + " | " + self.tabuleiro[7]+ " | " + self.tabuleiro[8] + " | " + self.tabuleiro[9] + " | " + "\n" +
      "| " + self.tabuleiro[10] + " | " + self.tabuleiro[11] + " | " + self.tabuleiro[12]+ " | " + self.tabuleiro[13] + " | " + self.tabuleiro[14] + " | " + "\n" +
      "| " + self.tabuleiro[15] + " | " + self.tabuleiro[16] + " | " + self.tabuleiro[17]+ " | " + self.tabuleiro[18] + " | " + self.tabuleiro[19] + " | " + "\n" +
      "| " + self.tabuleiro[20] + " | " + self.tabuleiro[21] + " | " + self.tabuleiro[22]+ " | " + self.tabuleiro[23] + " | " + self.tabuleiro[24] + " | " + "\n")

# Minimax com poda alfa-beta para melhorar a eficiência
def minimax(jogo, turno_max, jogador, profundidade_max=3, alfa=float("-inf"), beta=float("inf")):
    if jogo.venceu() or jogo.empate() or profundidade_max == 0:
        return jogo.calcular_utilidade(jogador)

    if turno_max:
        melhor = float("-inf")
        for movimento in jogo.jogos_validos():
            resultado = jogo.jogar(movimento)
            if resultado:  # Garantir que o movimento é válido
                valor = minimax(resultado, False, jogador, profundidade_max - 1, alfa, beta)
                melhor = max(melhor, valor)
                alfa = max(alfa, melhor)
                if beta <= alfa:
                    break  # Poda beta
        return melhor
    else:
        pior = float("inf")
        for movimento in jogo.jogos_validos():
            resultado = jogo.jogar(movimento)
            if resultado:  # Garantir que o movimento é válido
                valor = minimax(resultado, True, jogador, profundidade_max - 1, alfa, beta)
                pior = min(pior, valor)
                beta = min(beta, pior)
                if beta <= alfa:
                    break  # Poda alfa
        return pior

# Melhor jogada usando Minimax com aleatoriedade para evitar repetições
def melhor_jogada(jogo, profundidade=2):
    movimentos = jogo.jogos_validos()
    if not movimentos:  # Se não houver movimentos válidos
        return None

    # Adiciona aleatoriedade e evita jogadas repetidas
    if random.random() < 0.2:  # 20% de chance de fazer uma jogada aleatória
        return random.choice(movimentos)

    # Evita repetir a última jogada
    if jogo.ultima_jogada_agente in movimentos and len(movimentos) > 1:
        movimentos.remove(jogo.ultima_jogada_agente)

    # Avalia os movimentos e escolhe o melhor
    melhor_valor = float("-inf")
    melhores_movimentos = []

    for movimento in movimentos:
        resultado = jogo.jogar(movimento)
        if resultado:
            valor = minimax(resultado, False, jogo.turno(), profundidade)
            if valor > melhor_valor:
                melhor_valor = valor
                melhores_movimentos = [movimento]
            elif valor == melhor_valor:
                melhores_movimentos.append(movimento)

    # Escolhe aleatoriamente entre os movimentos de mesmo valor
    return random.choice(melhores_movimentos) if melhores_movimentos else movimentos[0]

# Mostra ajuda com regras do jogo
def mostrar_ajuda():
    print("\n=== REGRAS DO QUIXO ===")
    print("1. O tabuleiro é 5x5")
    print("2. Objetivo: formar uma linha de 5 peças do seu símbolo (horizontal, vertical ou diagonal)")
    print("3. Regras de movimento:")
    print("   - Só pode mover peças da borda")
    print("   - Só pode mover peças vazias ou suas peças")
    print("   - A peça é removida e empurrada do lado oposto")
    print("   - Sua peça sempre fica do lado para onde empurrou")
    print("4. Formato da jogada: origem destino (ex: 1,1 5,1)")
    print("======================\n")

# Execução principal
if __name__ == "__main__":
    # Usuário começa com 🔴, agente joga com ❌
    estado = Jogo(jogador='🔴')
    contador_turnos = 0
    max_turnos = 100  # Evita loops infinitos

    print("\n\n ❌🔴❌🔴❌🔴❌🔴 BEM-VINDO AO JOGO QUIXO ❌🔴❌🔴❌🔴❌🔴 \n\n")
    print("Você joga com 🔴 e o agente com ❌\n")
    print("🆘 Digite 'ajuda' para ver as regras do jogo")
    print("📄 Digite 'historico' para ver as últimas jogadas")
    print("🔚 Digite 'sair' para encerrar o jogo\n\n")

    while contador_turnos < max_turnos:
        estado.imprimir()

        if estado.venceu():
            vencedor = '🔴' if estado.jogador_atual == '❌' else '❌'
            print(f"🏆 VITÓRIA DO JOGADOR {vencedor}! 🏆")
            estado.historico.imprimir_historico()
            break
        elif estado.empate():
            print("🤝 EMPATE! 🤝")
            estado.historico.imprimir_historico()
            break

        if estado.turno() == "🔴":  # Turno do usuário
            while True:
                try:
                    entrada = input("Sua jogada (origem destino): ").strip().lower()

                    # Comandos especiais
                    if entrada == "ajuda":
                        mostrar_ajuda()
                        continue
                    elif entrada == "historico":
                        estado.historico.imprimir_historico()
                        continue
                    elif entrada == "sair":
                        print("Jogo encerrado. Até a próxima!")
                        exit()

                    origem, destino = entrada.split()
                    novo_estado = estado.jogar((origem, destino))

                    if novo_estado:
                        # Registra jogada no histórico
                        estado.historico.adicionar_jogada("🔴", origem, destino)
                        estado = novo_estado
                        break
                    else:
                        print("❌ Movimento inválido! Lembre-se das regras do Quixo.")
                        print("Só pode mover peças da borda, que sejam vazias ou suas.")
                except (ValueError, IndexError) as e:
                    print("⚠️ Formato inválido. Use: 'linha,coluna linha,coluna' (ex: 1,1 5,1)")
        else:  # Turno do agente
            print("\n🤖 Agente pensando...\n")
            mov = melhor_jogada(estado)
            if mov:
                origem, destino = mov
                print(f"\n🤖 Agente move de {origem} para {destino}\n")

                # Registra jogada no histórico
                estado.historico.adicionar_jogada("❌", origem, destino)

                # Guarda a jogada do agente para evitar repetição
                estado.ultima_jogada_agente = mov

                # Executa a jogada
                estado = estado.jogar(mov)
            else:
                print("Agente não conseguiu encontrar um movimento válido. Fim de jogo.")
                break

        contador_turnos += 1

    if contador_turnos >= max_turnos:
        print("Jogo encerrado por número excessivo de turnos.")
        estado.historico.imprimir_historico()



 ❌🔴❌🔴❌🔴❌🔴 BEM-VINDO AO JOGO QUIXO ❌🔴❌🔴❌🔴❌🔴 


Você joga com 🔴 e o agente com ❌

🆘 Digite 'ajuda' para ver as regras do jogo
📄 Digite 'historico' para ver as últimas jogadas
🔚 Digite 'sair' para encerrar o jogo


| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 

Sua jogada (origem destino): ajuda

=== REGRAS DO QUIXO ===
1. O tabuleiro é 5x5
2. Objetivo: formar uma linha de 5 peças do seu símbolo (horizontal, vertical ou diagonal)
3. Regras de movimento:
   - Só pode mover peças da borda
   - Só pode mover peças vazias ou suas peças
   - A peça é removida e empurrada do lado oposto
   - Sua peça sempre fica do lado para onde empurrou
4. Formato da jogada: origem destino (ex: 1,1 5,1)

Sua jogada (origem destino): 1,1 5,1
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| 🔴 | ⬜ | ⬜ | ⬜ | ⬜ | 


🤖 Agente pensando...


🤖 Agente move de 5,5 para 5,1

| ⬜ | ⬜ | ⬜ | ⬜ | ⬜ | 
| ⬜ | ⬜ 

KeyboardInterrupt: Interrupted by user